In [8]:
# import libraries

import matplotlib.pyplot as plt

import cartopy.crs as ccrs
#from cartopy.util import add_cyclic_point

import xarray as xr

import numpy as np


from pathlib import Path
import sys

In [9]:
path_home = Path.home()
path_home

PosixPath('/home/giodefa')

In [10]:
def searching_model_path(name: str):
    """Insert the name string of the model so i can reach for you"""
    try:
        path_home = Path.home()
        path_directory = path_home / 'Desktop/Unimib/Data Science/Second Year/Physics and environmental data lab/Environmental/Materiale Corso/Model Folder'
        path_model_ls_cm6a = path_directory / name
        if path_model_ls_cm6a.exists() == False:
            sys.exit()
        return path_model_ls_cm6a
    except SystemExit:
        print('Insert string and valid model name')

In [11]:
class Searching_Model_Path:

    def __init__(self, searching_model_path=None):
        self.searching_model_path = searching_model_path
    
    def searching_model_path(name: str):
        """Insert the name string of the model so i can reach for you"""
        try:
            path_home = Path.home()
            path_directory = path_home / 'Desktop/Unimib/Data Science/Second Year/Physics and environmental data lab/Environmental/Materiale Corso/Model Folder'
            path_model_ls_cm6a = path_directory / name
            if path_model_ls_cm6a.exists() == False:
                sys.exit()
            return path_model_ls_cm6a
        except SystemExit:
            print('Insert string and valid model name')


In [12]:
Searching_Model_Path.searching_model_path('ts_Amon_MIROC6_historical_r1i1p1f1_gn_185001-201412.nc')

PosixPath('/home/giodefa/Desktop/Unimib/Data Science/Second Year/Physics and environmental data lab/Environmental/Materiale Corso/Model Folder/ts_Amon_MIROC6_historical_r1i1p1f1_gn_185001-201412.nc')

In [18]:
path_model_ls_cm6a =  searching_model_path('ts_Amon_MIROC6_historical_r1i1p1f1_gn_185001-201412.nc')

In [19]:
d1 = xr.open_dataset(path_model_ls_cm6a)
d1

ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
http://xarray.pydata.org/en/stable/user-guide/io.html 
http://xarray.pydata.org/en/stable/getting-started-guide/installing.html

In [2]:
#- Open & explore the netCDF file for model 1; extract model name

modfile1='./ts_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc'
#modfile1='./ts_Amon_MRI-ESM2-0_historical_r1i1p1f1_gn_185001-201412.nc'

m_variable='./ts_Amon_'
m_case='_historical_r1i1p1f1_gr_185001-201412.nc'
model1=modfile1[len(m_variable):-len(m_case)]

d1 = xr.open_dataset(modfile1)
d1

FileNotFoundError: [Errno 2] No such file or directory: b'/home/giodefa/Desktop/Unimib/Data Science/Second Year/Physics and environmental data lab/Environmental/Materiale Corso/Esercitazione 2/ts_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc'